In [20]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from anndata import AnnData
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from pathlib import Path

In [3]:
def get_proportions(meta):
    return (meta
     .groupby(['sample_id', 'cell_types'])[['sample_id']]
     .count()
     .rename({'sample_id': 'n'}, axis=1)
     .reset_index()
     .assign(sc=lambda x: x['n'] / x.groupby('sample_id')['n'].transform('sum'))
     .drop('n', axis=1)
    )

In [21]:
current_folder = globals()['_dh'][0]
path_ann = current_folder / ".." / ".." / "data" / "prc" / "sc"

In [23]:
adata = sc.read_h5ad(path_ann / "annotated_cellranger.h5ad")

In [26]:
adata.obs.cell_types.value_counts()

cell_types
Oligos        72381
Neurons       13993
Microglia     12531
Astros        11982
OPC            5453
Asrtos_r       5242
Endothelia     3777
Macros         1809
T_cells        1310
Astros_n        901
B_cells         702
Stroma          492
Astros_c        457
Name: count, dtype: int64

In [6]:
current_folder = globals()['_dh'][0]
path_ann = current_folder / ".." / ".." / "data" / "prc" / "sc"
annotated_meta = {method: sc.read_h5ad(path_ann / f"annotated_{method}_mod.h5ad").obs for method in ["cellbender", "cellranger"]}

In [16]:
annotated_meta["cellranger"].groupby(["Condition", "cell_types"]).count().reset_index()

,Condition,cell_types,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,doublet_score,predicted_doublet,diss_score,...,sample_id,lesion_type,Age,Sex,RIN,Batch,visium,snRNA-seq,batch,leiden
0,Control,Astros,2374,2374,2374,2374,2374,2374,2374,2374,...,2374,2374,2374,2374,2374,2374,2374,2374,2374,2374
1,Control,Astros_c,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45
2,Control,Astros_r,736,736,736,736,736,736,736,736,...,736,736,736,736,736,736,736,736,736,736
3,Control,B_cells,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,Control,Endothelia,555,555,555,555,555,555,555,555,...,555,555,555,555,555,555,555,555,555,555
5,Control,Macrophages,84,84,84,84,84,84,84,84,...,84,84,84,84,84,84,84,84,84,84
6,Control,Microglia,2039,2039,2039,2039,2039,2039,2039,2039,...,2039,2039,2039,2039,2039,2039,2039,2039,2039,2039
7,Control,Neurons,2074,2074,2074,2074,2074,2074,2074,2074,...,2074,2074,2074,2074,2074,2074,2074,2074,2074,2074
8,Control,OPC,1390,1390,1390,1390,1390,1390,1390,1390,...,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390
9,Control,Oligos,21176,21176,21176,21176,21176,21176,21176,21176,...,21176,21176,21176,21176,21176,21176,21176,21176,21176,21176


In [19]:
annotated_meta["cellbender"].groupby(["Condition", "cell_types"]).count().reset_index()[["Condition", "cell_types", "n_genes"]].sort_values("n_genes", ascending=False)

,Condition,cell_types,n_genes
17,MS,Oligos,61555
7,Control,Oligos,26842
10,MS,Astros,13983
14,MS,Microglia,10735
15,MS,Neurons,9900
16,MS,OPC,3894
13,MS,Endothelia,2853
0,Control,Astros,2805
4,Control,Microglia,2022
5,Control,Neurons,1549


In [18]:
annotated_meta["cellranger"].groupby(["Condition", "cell_types"]).count().reset_index()[["Condition", "cell_types", "n_genes"]].sort_values("n_genes", ascending=False)

,Condition,cell_types,n_genes
21,MS,Oligos,51918
9,Control,Oligos,21176
19,MS,Neurons,11471
18,MS,Microglia,11071
12,MS,Astros,10519
14,MS,Astros_r,4645
20,MS,OPC,4024
16,MS,Endothelia,3213
0,Control,Astros,2374
7,Control,Neurons,2074
